In [1]:
import ssl
import requests
import time
import pandas as pd
import csv
from lxml import html
import os
import pickle

In [2]:
def scrape_reviews(asins):
    ratings_dict = {}
    reviews_list = []
    reviews_df = pd.DataFrame()
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
    headers = {'User-Agent' : user_agent}
    XPATH_REVIEWS = '//div[@data-hook="review"]'
    XPATH_REVIEW_RATING = './/i[@data-hook="review-star-rating"]//text()'
    XPATH_REVIEW_HEADER = './/a[@data-hook="review-title"]//span[@class]//text()'
    XPATH_REVIEW_AUTHOR = './/div[@class="a-profile-content"]//text()'
    XPATH_REVIEW_DATE = './/span[@data-hook="review-date"]//text()'
    XPATH_REVIEW_BODY = './/span[@data-hook="review-body"]//span[@class]//text()'
    XPATH_REVIEW_HELPFUL = './/span[@data-hook="helpful-vote-statement"]//text()'
    
    p_num = 1
    for asin in asins:
        while True:
#avp_only_reviews =  for only purchase the product
#all_reviews = for all review even user that not purchase the product
            review = ('avp_only_reviews')
            print ('Scraping review page nr. {}'.format(p_num))
            url = 'https://www.amazon.com/product-reviews/' + asin + '?pageNumber=' + str(p_num) + '&sortBy=recent' + '&ie=UTF8' + '&reviewerType=' + (review)
            print(url)
            page = requests.get(url, headers=headers)
            
            page_response = page.text.encode('utf-8')
            
            parser = html.fromstring(page_response)
            reviews = parser.xpath(XPATH_REVIEWS)
            if not len(reviews) > 0:
                break
            for review in reviews:
                raw_review_author = review.xpath(XPATH_REVIEW_AUTHOR)
                raw_review_rating = review.xpath(XPATH_REVIEW_RATING)
                raw_review_header = review.xpath(XPATH_REVIEW_HEADER)
                raw_review_date = review.xpath(XPATH_REVIEW_DATE)
                raw_review_body = review.xpath(XPATH_REVIEW_BODY)
                raw_review_helpful = review.xpath(XPATH_REVIEW_HELPFUL)
                
                review_dict = {
                    'review_text': raw_review_body,
                    'review_posted_date': raw_review_date,
                    'review_header': raw_review_header,
                    'review_rating': raw_review_rating,
                    'review_helpful': raw_review_helpful,
                    'review_author': raw_review_author
                }
                reviews_df = reviews_df.append(review_dict, ignore_index=True)
            p_num += 1
            if p_num == 495 ++ 1:
                break
        return reviews_df

In [3]:
filename = "pass"
filenamepickle = 'ipad.pickle'
filenamejson = 'ipad.json'
filenamecsv = 'ipad.csv'

asins = ['B07CZMMQPM'] # Apple iPad (Wi-Fi, 32GB) - Gold (Latest Model)

if os.path.isfile(filename):
    print ('Loading reviews from disk')
    reviews_df = pd.read_pickle(filename)
    print ('Loaded {} reviews'.format(len(reviews_df)))
else:
    print ('Scraping reviews')
    reviews_df = scrape_reviews(asins)
    print ('Scraped {} reviews'.format(len(reviews_df)))
    print ('Saving reviews to disk')
    reviews_df.to_json(filenamejson)
    reviews_df.to_pickle(filenamepickle)
    reviews_df.to_csv(filenamecsv)

Scraping reviews
Scraping review page nr. 1
https://www.amazon.com/product-reviews/B07CZMMQPM?pageNumber=1&sortBy=recent&ie=UTF8&reviewerType=avp_only_reviews
Scraped 0 reviews
Saving reviews to disk


In [ ]:
#reviews_df= pd.read_json(filenamejson_biasa)
reviews_df.head()

In [ ]:
for col in reviews_df.columns:
    reviews_df[col] = reviews_df[col].apply(lambda x: '\n'.join(x))

reviews_df['review_helpful'] = (reviews_df['review_helpful']
                                .str.replace('One', '1')
                                .str.replace(r'[^0-9]', ''))
reviews_df['review_helpful'].loc[reviews_df['review_helpful'] == ''] = '0'
reviews_df['review_helpful'] = reviews_df['review_helpful'].astype(int)

reviews_df['review_posted_date'] = pd.to_datetime(reviews_df['review_posted_date']
                                                  .str.strip('on'))

reviews_df['review_rating'] = reviews_df['review_rating'].str.strip('out of 5 stars').astype(float)
reviews_df.loc[reviews_df['review_rating'] == 0, 'review_rating'] = 5

reviews_df['review_length'] = reviews_df['review_text'].apply(lambda x: len(x))

reviews_df.drop_duplicates(inplace=True)
reviews_df.index = pd.RangeIndex(start=1, stop =len(reviews_df) + 1, step=1)

In [ ]:
reviews_df

In [ ]:
reviews_df.to_csv("s9_processed.csv")
reviews_df.to_json("s9_processed.json")
reviews_df.to_pickle("s9_processed.pickle")

In [ ]:
#gak kepake
#reviews_df.to_json(filenamejson_index, orient='index')
#    reviews_df.to_json(filenamejson_split, orient='split')
#    reviews_df.to_json(filenamejson_records, orient='records')
#    reviews_df.to_json(filenamejson_index, orient='index')
#    reviews_df.to_json(filenamejson_columns, orient='columns')
#filenamejson_index = 's8_index.json'
#filenamejson_split = 's8_split.json'
#filenamejson_records = 's8_records.json'
#filenamejson_index = 's8_index.json'
#filenamejson_columns = 's8_columns.json'
#filenamejson_values = 's8_values.json'
#filenamejson_table = 's8_table.json'
#filenamejson_biasa = 's8_biasa.json'

In [ ]:
reviews_df= pd.read_pickle("ipad.pickle")
reviews_df.head(100+1)

In [ ]:
for col in reviews_df.columns:
    reviews_df[col] = reviews_df[col].apply(lambda x: '\n'.join(x))

reviews_df['review_helpful'] = (reviews_df['review_helpful']
                                .str.replace('One', '1')
                                .str.replace(r'[^0-9]', ''))
reviews_df['review_helpful'].loc[reviews_df['review_helpful'] == ''] = '0'
reviews_df['review_helpful'] = reviews_df['review_helpful'].astype(int)

reviews_df['review_posted_date'] = pd.to_datetime(reviews_df['review_posted_date']
                                                  .str.strip('on'))

reviews_df['review_rating'] = reviews_df['review_rating'].str.strip('out of 5 stars').astype(float)
reviews_df.loc[reviews_df['review_rating'] == 0, 'review_rating'] = 5

reviews_df['review_length'] = reviews_df['review_text'].apply(lambda x: len(x))

reviews_df.drop_duplicates(inplace=True)
reviews_df.index = pd.RangeIndex(start=1, stop =len(reviews_df) + 1, step=1)

In [ ]:
reviews_df.to_pickle("ipad_processed.pickle")
reviews_df.to_csv("ipad_processed.txt")


In [ ]:
reviews_df.head(100)